# Vector Space Model

In [12]:
import pandas as pd
import numpy as np
import os
import nltk
import collections

In [13]:
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

In [31]:
class VSM:
    
    def __init__(self, query):
        self.query = query
        self.tf_idf = pd.DataFrame()
        self.idf = []
        self.docs = []
        self.terms = []
        
    def buildVectorSpace(self):
        files = os.listdir('Cranfield Data Set')[:100]
        n = len(files)
        for i in range(len(files)):
            with open(f"Cranfield Data Set/{files[i]}") as f:
                text = f.read()
                text = text.lower()
                self.docs.append(text)
                self.terms.extend(text.split()) 
        
        self.terms = sorted(list(set(self.terms)))
        self.terms = list(set(self.terms).difference(stopWords))
        self.idf = [0 for i in range(len(self.terms))]
        self.tf_idf = pd.DataFrame([[0 for i in range(len(self.terms))] for j in range(n)])
        self.tf_idf.index = [f'Doc{i}' for i in range(1, n+1)]
        self.tf_idf.columns = self.terms
        
        for i in range(len(self.docs)):
            for j in range(len(self.terms)):
                if self.docs[i].count(self.terms[j]) != 0:
                    tf = 1 + math.log10(self.docs[i].count(self.terms[j]))
                else:
                    tf = 0
                self.tf_idf.iloc[i, j] = tf
        for i in range(len(self.terms)):
            self.idf[i] = math.log10(len(self.docs) / (self.tf_idf.iloc[:, i] != 0).sum())
            self.tf_idf.iloc[:, i] *= self.idf[i]

    def cosineSimilarity(self, vec1, vec2):
        dotProduct = vec1.dot(vec2)
        normVec1 = np.linalg.norm(vec1)
        normVec2 = np.linalg.norm(vec2)
        
        if normVec1 != 0 and normVec2 != 0:
            return dotProduct / (normVec1 * normVec2)
        else:
            return 0

    def queryVector(self):
        cosineSim = []
        queryTerms = self.query.lower().split()
        queryVec = pd.Series([0] * len(self.terms), index=self.terms)
        
        for qTerm in queryTerms:
            if qTerm in self.terms:
                queryVec[qTerm] += 1
        
        for i in range(len(self.docs)):
            val = self.cosineSimilarity(queryVec, self.tf_idf.iloc[i])
            cosineSim.append(val)
        cosineSim = cosineSim[:100]
        ranks = [sorted(cosineSim, reverse=True).index(x) for x in cosineSim]
        print("Ranked documents: ")
        for i in range(len(self.docs)):
            print(ranks.index(i))
            print(self.docs[ranks.index(i)])
            print()

In [32]:
vsm = VSM(input("Enter the query: "))
vsm.buildVectorSpace()
vsm.queryVector()

Enter the query:  what are the structural and aeroelastic problems associated with flight of high speed aircraft


Ranked documents: 
11
the dominating factors in structural design of high speed aircraft are thermal and aeroelastic in origin   the subject matter is concerned largely with a discussion of these factors and their interrelation with one another   a summary is presented of some of the analytical and experimental tools available to aeronautical engineers to meet the demands of high speed flight upon aircraft structures   the state of the art with respect to heat transfer from the boundary layer into the structure modes of failure under combined load as well as thermal inputs and acrothermoelasticity is discussed   methods of attacking and alleviating structural and aeroelastic problems of high speed flight are summarized   finally some avenues of fundamental research are suggested 

50
the problem of investigating the simultaneous effects of transient aerodynamic heating and external loads on aircraft structures for the purpose of determining the ability of the structure to withstand fli

ValueError: 51 is not in list